In [ ]:
#############################################################################
### DEBUGGED AND EDITED
#############################################################################

In [2]:
import numpy as np
import pandas as pd
import os, glob
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import community as community_louvain

In [ ]:

data_dir = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation6"
pattern     = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")

# Define the output directory for network plots
output_dir = os.path.join(data_dir, "network_graphs")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist



In [14]:
#########################################################
### Spatial Network Plots and Entropy Calculations ###
##########################################################

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# 1) find & sort your files
data_dir = r"C:\Users\myPC\Agroforestal\Discover Networks\Simulations\Simulation6"

pattern     = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
#pattern     = os.path.join(data_dir, "weighted_matrix_combined_tick_*.csv")
files       = glob.glob(pattern)
files.sort(key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))

# Define the output directory for network plots
output_dir = os.path.join(data_dir, "network_plots2")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# helper to pull tick number out of filename
def extract_tick(fp):
    return int(os.path.basename(fp).split('_')[-1].split('.')[0])

# the core loop
ticks      = []
entropies  = []

for fp in files:
    tick = extract_tick(fp)
    ticks.append(tick)

    # --- read & entropy ---
    df = pd.read_csv(fp)
    adj = df.pivot(index='source', columns='target', values='weight').fillna(0)
    P   = adj.div(adj.sum(axis=1), axis=0)
    p   = P.values.flatten()
    p   = p[p > 0]
    p  /= p.sum()
    H   = -(p * np.log2(p)).sum()
    entropies.append(H)
    print(f"tick {tick:3d}: H = {H:.4f} bits")

    # --- build graph & extract real positions ---
    G = nx.DiGraph()
    pos = {}
    for _, row in df.iterrows():
        src, tgt = int(row.source), int(row.target)
        w        = row.weight

        # add nodes + record their true x/y
        if src not in G:
            G.add_node(src)
            pos[src] = (row.source_x, row.source_y)
        if tgt not in G:
            G.add_node(tgt)
            pos[tgt] = (row.target_x, row.target_y)

        # add the directed edge
        G.add_edge(src, tgt, weight=w)

    # --- visualize with reversed cmap ---
    fig, ax = plt.subplots(figsize=(8, 6))
    nx.draw_networkx_nodes(
        G, pos, ax=ax,
        node_size=32,
        node_color='green',
        alpha=0.8
    )

    # edge styling
    weights = [abs(d['weight']) * 2 for _,_,d in G.edges(data=True)]
    colors  = [d['weight']        for _,_,d in G.edges(data=True)]
    M       = max(abs(min(colors)), abs(max(colors)))

    nx.draw_networkx_edges(
        G, pos, ax=ax,
        width=weights,
        edge_color=colors,
        edge_cmap=plt.cm.RdBu,    # <<-- standard RdBu: low→red, high→blue
        edge_vmin=-M,             # negative end
        edge_vmax=+M,             # positive end
        alpha=0.6
    )

    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.RdBu_r,
        norm=plt.Normalize(vmin=-M, vmax=+M)
    )
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label("Edge weight")

    ax.set_title(f"Network at tick {tick}")
    ax.axis('off')
    plt.tight_layout()
    # save the figure
    outpath = os.path.join(output_dir, f"network_tick_{tick:03d}.png")
    plt.savefig(outpath, dpi=300)
    print(f"  → saved {outpath}")
    plt.close(fig)

# --- final H vs tick plot ---
plt.figure(figsize=(6,4))
plt.plot(ticks, entropies, '-o')
plt.xlabel("Tick")
plt.ylabel("Shannon entropy (bits)")
plt.tight_layout()
plt.savefig("entropy_vs_tick.png", dpi=300)

plt.figure(figsize=(6,4))
plt.plot(ticks, entropies, '-o')
plt.xlabel("Tick")
plt.ylabel("Shannon entropy (bits)")
plt.tight_layout()

outpath = os.path.join(output_dir, f"Shannon entropy.png")
plt.savefig(outpath, dpi=300)
print(f"  → saved {outpath}")
plt.close(fig)


In [12]:
import numpy as np
import pandas as pd
import glob, os
from scipy.optimize import brentq  # or newton, fsolve
import matplotlib.pyplot as plt


In [ ]:
data_dir = r"C:\Users\myPCAgroforestal\Discover Networks\Simulations\Simulation6"
pattern     = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
#pattern     = os.path.join(data_dir, "weighted_matrix_combined_tick_*.csv")

# Define the output directory for network plots
output_dir = os.path.join(data_dir, "network_graphs")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist



In [ ]:
def visualize_network(G, layout='spring', node_color_map=None,
                      title="Network Visualization", max_nodes=1000):
    """
    Create a visualization of the network
    """
    print(f"Visualizing network with {G.number_of_nodes()} nodes...")
    
    if G.number_of_nodes() > max_nodes:
        print(f"Network too large for visualization, sampling {max_nodes} nodes...")
        nodes_sample = list(G.nodes())[:max_nodes]
        G = G.subgraph(nodes_sample)
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Choose layout …
    if layout == 'spring':
        pos = nx.spring_layout(G, seed=42)
    elif layout == 'circular':
        pos = nx.circular_layout(G)
    elif layout == 'kamada_kawai':
        pos = nx.kamada_kawai_layout(G)
    else:
        pos = nx.spring_layout(G, seed=42)
    
    # 1) draw every node in green
    nx.draw_networkx_nodes(
        G, pos, ax=ax,
        node_size=50,
        node_color='green',      # ← all nodes green
        alpha=0.8
    )
    
    # prepare edge widths + colors
    edge_weights = [abs(G[u][v]['weight']) * 3 for u, v in G.edges()]
    edge_colors  = [G[u][v]['weight'] for u, v in G.edges()]
    
    # 2) flip the color‐mapping so negatives are red and positives are blue
    M = max(abs(min(edge_colors)), abs(max(edge_colors)))
    # by swapping vmin/vmax we invert the mapping
    edges = nx.draw_networkx_edges(
        G, pos, ax=ax,
        width=edge_weights,
        alpha=0.5,
        edge_color=edge_colors,
        edge_cmap=plt.cm.RdBu,
        edge_vmin=+M,             # ← positive side now maps to low end (blue)
        edge_vmax=-M              # ← negative side now maps to high end (red)
    )
    
    # single colorbar
    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.RdBu,
        norm=plt.Normalize(vmin=+M, vmax=-M)
    )
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label('Edge weight')
    
    ax.set_title(title)
    ax.axis('off')
    plt.tight_layout()
    outpath = os.path.join(output_dir, f"network_tick_{tick:03d}.png")
    plt.savefig(outpath, dpi=300)
    print(f"  → saved {outpath}")
    plt.close(fig)


In [ ]:
###################################################################
### EDITED and with DEBUGGING loops
###################################################################

In [ ]:
data_dir = r"C:\Users\myPCAgroforestal\Discover Networks\Simulations\Simulation6"
pattern     = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
#pattern     = os.path.join(data_dir, "weighted_matrix_combined_tick_*.csv")

# Define the output directory for network plots
output_dir = os.path.join(data_dir, "network_plots2")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist



In [ ]:
import os, glob
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Enable interactive plotting
plt.ion()

# 1) find & sort your files
data_dir = r"C:\Users\myPCAgroforestal\Discover Networks\Simulations\Simulation6"

# Define output directory for plots
output_dir = os.path.join(data_dir, "network_plots2")
os.makedirs(output_dir, exist_ok=True)

print(f"Looking for files in: {data_dir}")
pattern = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
#pattern     = os.path.join(data_dir, "weighted_matrix_combined_tick_*.csv")
print(f"Search pattern: {pattern}")

files = glob.glob(pattern)
print(f"Found {len(files)} files")

if not files:
    print("ERROR: No files found!")
    print("Available files in directory:")
    all_files = os.listdir(data_dir)
    csv_files = [f for f in all_files if f.endswith('.csv')]
    for f in csv_files[:10]:  # Show first 10 CSV files
        print(f"  {f}")
    exit()

# Sort files by tick number
files.sort(key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))
print("Processing files:")
for f in files[:5]:  # Show first 5 files
    print(f"  {os.path.basename(f)}")

# helper to pull tick number out of filename
def extract_tick(fp):
    return int(os.path.basename(fp).split('_')[-1].split('.')[0])

# the core loop
ticks = []
entropies = []

print(f"\nProcessing {len(files)} files...")

for i, fp in enumerate(files):
    tick = extract_tick(fp)
    ticks.append(tick)
    
    print(f"\nProcessing file {i+1}/{len(files)}: tick {tick}")
    
    try:
        # --- read & entropy ---
        df = pd.read_csv(fp)
        print(f"  Loaded {len(df)} rows")
        print(f"  Columns: {list(df.columns)}")
        
        # Check if we have the required columns
        required_cols = ['source', 'target', 'weight', 'source_x', 'source_y', 'target_x', 'target_y']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            print(f"  WARNING: Missing columns: {missing_cols}")
            # Try alternative column names
            alt_mapping = {
                'source_x': ['source_x', 'source-x', 'x1'],
                'source_y': ['source_y', 'source-y', 'y1'],
                'target_x': ['target_x', 'target-x', 'x2'],
                'target_y': ['target_y', 'target-y', 'y2']
            }
            for std_name, alternatives in alt_mapping.items():
                if std_name in missing_cols:
                    for alt in alternatives:
                        if alt in df.columns:
                            df[std_name] = df[alt]
                            print(f"  Mapped {alt} -> {std_name}")
                            break
        
        # Create adjacency matrix for entropy calculation
        adj = df.pivot(index='source', columns='target', values='weight').fillna(0)
        print(f"  Adjacency matrix shape: {adj.shape}")
        
        # Calculate probability matrix
        row_sums = adj.sum(axis=1)
        # Only consider rows with non-zero sums
        valid_rows = row_sums > 0
        if valid_rows.sum() == 0:
            print(f"  WARNING: No valid rows for tick {tick}")
            continue
            
        P = adj[valid_rows].div(row_sums[valid_rows], axis=0)
        p = P.values.flatten()
        p = p[p > 0]
        
        if len(p) == 0:
            print(f"  WARNING: No positive probabilities for tick {tick}")
            continue
            
        p /= p.sum()
        H = -(p * np.log2(p)).sum()
        entropies.append(H)
        print(f"  tick {tick:3d}: H = {H:.4f} bits")
        
        # --- build graph & extract real positions ---
        G = nx.DiGraph()
        pos = {}
        
        for _, row in df.iterrows():
            src, tgt = int(row.source), int(row.target)
            w = row.weight
            
            # add nodes + record their true x/y
            if src not in G:
                G.add_node(src)
                pos[src] = (row.source_x, row.source_y)
            if tgt not in G:
                G.add_node(tgt)
                pos[tgt] = (row.target_x, row.target_y)
            
            # add the directed edge
            G.add_edge(src, tgt, weight=w)
        
        print(f"  Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
        
        # --- visualize with reversed cmap ---
        plt.close('all')  # Close any existing figures
        
        fig, ax = plt.subplots(figsize=(8, 6))
        
        # Draw nodes
        nx.draw_networkx_nodes(
            G, pos, ax=ax,
            node_size=32,
            node_color='green',
            alpha=0.8
        )
        
        # edge styling
        edge_data = list(G.edges(data=True))
        if edge_data:  # Only if we have edges
            weights = [abs(d['weight']) * 2 for _, _, d in edge_data]
            colors = [d['weight'] for _, _, d in edge_data]
            
            if colors:  # Only if we have color data
                M = max(abs(min(colors)), abs(max(colors)))
                
                nx.draw_networkx_edges(
                    G, pos, ax=ax,
                    width=weights,
                    edge_color=colors,
                    edge_cmap=plt.cm.RdBu,    # FIXED: Use RdBu_r for red=negative, blue=positive
                    edge_vmin=-M,
                    edge_vmax=+M,
                    alpha=0.6
                )
                
                # Add colorbar
                sm = plt.cm.ScalarMappable(
                    cmap=plt.cm.RdBu,  # FIXED: Match the edge colormap
                    norm=plt.Normalize(vmin=-M, vmax=+M)
                )
                sm.set_array([])
                cbar = fig.colorbar(sm, ax=ax)
                cbar.set_label("Edge weight")
        
        ax.set_title(f"Network at tick {tick}")
        ax.axis('off')
        plt.tight_layout()
        
        # Save the figure
        outname = os.path.join(output_dir, f"network_tick_{tick:03d}.png")
        plt.savefig(outname, dpi=300, bbox_inches='tight')
        print(f"  Saved: {outname}")
        
        # Show the figure
        plt.show()
        plt.pause(2)  # Pause for 2 seconds
        
    except Exception as e:
        print(f"  ERROR processing tick {tick}: {e}")
        import traceback
        traceback.print_exc()
        continue

# FIXED: Move the entropy plot outside the loop
print(f"\nCreating entropy plot with {len(ticks)} data points...")

if ticks and entropies:
    plt.figure(figsize=(6, 4))
    plt.plot(ticks, entropies, '-o')
    plt.xlabel("Tick")
    plt.ylabel("Shannon entropy (bits)")
    plt.title("Shannon Entropy vs Time")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    entropy_plot_path = os.path.join(output_dir, "entropy_vs_tick.png")
    plt.savefig(entropy_plot_path, dpi=300, bbox_inches='tight')
    print(f"Saved entropy plot: {entropy_plot_path}")
    plt.show()

    
else:
    print("No data to plot entropy!")

plt.ioff()  # Turn off interactive mode
print("\nProcessing complete!")

In [22]:
#######################################################################################
### Zooming in on specific ticks for comparison - Experimental
#######################################################################################


import os, numpy as np, pandas as pd, networkx as nx, matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def load_weighted_spatial_csv(fp):
    df = pd.read_csv(fp)
    need = ['source','target','weight','source_x','source_y','target_x','target_y']
    miss = [c for c in need if c not in df.columns]
    if miss:
        alt = {
            'source_x':['source_x','source-x','x1'],
            'source_y':['source_y','source-y','y1'],
            'target_x':['target_x','target-x','x2'],
            'target_y':['target_y','target-y','y2'],
        }
        for std, alts in alt.items():
            if std in miss:
                for a in alts:
                    if a in df.columns:
                        df[std] = df[a]
                        break
        still = [c for c in need if c not in df.columns]
        if still:
            raise ValueError(f"Missing columns after mapping: {still}")
    return df

def build_graph_from_df(df):
    G = nx.DiGraph()
    pos = {}
    for _, r in df.iterrows():
        u, v, w = int(r.source), int(r.target), float(r.weight)
        if u not in G:
            G.add_node(u); pos[u] = (float(r.source_x), float(r.source_y))
        if v not in G:
            G.add_node(v); pos[v] = (float(r.target_x), float(r.target_y))
        G.add_edge(u, v, weight=w)
    return G, pos

def filter_edges(G, mode=None, k=5, abs_thresh=None):
    """Return a list of edges (u,v) after filtering.
       mode: None | 'topk_out' | 'topk_in' — top-k by |weight| per node.
       abs_thresh: keep edges with |weight| >= abs_thresh (applied last)."""
    edges = list(G.edges(data=True))
    if mode in ('topk_out','topk_in'):
        keep = set()
        if mode=='topk_out':
            by_node = {u:[] for u in G.nodes()}
            for u,v,d in edges: by_node[u].append((u,v,abs(d['weight'])))
            for u, lst in by_node.items():
                for u,v,_ in sorted(lst, key=lambda t:t[2], reverse=True)[:k]:
                    keep.add((u,v))
        else:  # topk_in
            by_node = {v:[] for v in G.nodes()}
            for u,v,d in edges: by_node[v].append((u,v,abs(d['weight'])))
            for v, lst in by_node.items():
                for u,v,_ in sorted(lst, key=lambda t:t[2], reverse=True)[:k]:
                    keep.add((u,v))
        edges = [(u,v,d) for u,v,d in edges if (u,v) in keep]
    if abs_thresh is not None:
        edges = [(u,v,d) for u,v,d in edges if abs(d['weight']) >= abs_thresh]
    return edges

def plot_spatial_network(
    fp, out_png, node_size=24, arrows=True,
    cmap='RdBu',  # red = negative, blue = positive
    edge_scale=2.0, mode=None, k=5, abs_thresh=None,
    title=None, dpi=300
):
    
    df = load_weighted_spatial_csv(fp)
    G, pos = build_graph_from_df(df)
    edata = filter_edges(G, mode=mode, k=k, abs_thresh=abs_thresh)
    ws = np.array([d['weight'] for _,_,d in edata]) if edata else np.array([0.0])
    M = max(abs(ws.min()), abs(ws.max())) if ws.size else 1.0

    fig, ax = plt.subplots(figsize=(9,7))
    nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='forestgreen', alpha=0.9, ax=ax)
    if edata:
        widths = [edge_scale*abs(d['weight']) for _,_,d in edata]
        colors = [d['weight'] for _,_,d in edata]
        nx.draw_networkx_edges(
            G, pos, edgelist=[(u,v) for u,v,_ in edata],
            width=widths, edge_color=colors, edge_cmap=plt.cm.get_cmap(cmap),
            edge_vmin=-M, edge_vmax=+M, alpha=0.6,
            arrows=arrows, arrowsize=8, ax=ax
        )
        sm = plt.cm.ScalarMappable(cmap=plt.cm.get_cmap(cmap), norm=plt.Normalize(vmin=-M, vmax=+M))
        sm.set_array([]); cbar = fig.colorbar(sm, ax=ax, fraction=0.03, pad=0.03)
        cbar.set_label("Edge weight", rotation=90)

    ax.set_title(title or os.path.basename(fp))
    ax.set_aspect('equal', adjustable='box'); ax.axis('off')
    fig.tight_layout(); fig.savefig(out_png, dpi=dpi, bbox_inches='tight')
    plt.close(fig)
    return out_png

def plot_zoom(fp, out_png, region, **kwargs):
    """region = (xmin, xmax, ymin, ymax) in the same units as source/target coords."""
    df = load_weighted_spatial_csv(fp); G, pos = build_graph_from_df(df)
    xmin,xmax,ymin,ymax = region
    nodes_in = [n for n,(x,y) in pos.items() if xmin<=x<=xmax and ymin<=y<=ymax]
    H = G.subgraph(nodes_in).copy()
    edata = [(u,v,d) for u,v,d in H.edges(data=True)]
    ws = np.array([d['weight'] for _,_,d in edata]) if edata else np.array([0.0])
    M = max(abs(ws.min()), abs(ws.max())) if ws.size else 1.0

    fig, ax = plt.subplots(figsize=(7,6))
    nx.draw_networkx_nodes(H, pos, nodelist=nodes_in, node_size=28, node_color='forestgreen', alpha=0.95, ax=ax)
    if edata:
        widths = [2.5*abs(d['weight']) for _,_,d in edata]
        colors = [d['weight'] for _,_,d in edata]
        nx.draw_networkx_edges(
            H, pos, edgelist=[(u,v) for u,v,_ in edata],
            width=widths, edge_color=colors, edge_cmap=plt.cm.RdBu,
            edge_vmin=-M, edge_vmax=+M, alpha=0.75, arrows=True, arrowsize=10, ax=ax
        )
        sm = plt.cm.ScalarMappable(cmap=plt.cm.RdBu, norm=plt.Normalize(vmin=-M, vmax=+M))
        sm.set_array([]); fig.colorbar(sm, ax=ax, fraction=0.035, pad=0.035, label="Edge weight")

    ax.add_patch(Rectangle((xmin,ymin), xmax-xmin, ymax-ymin, fill=False, lw=1.2, ls='--', ec='black'))
    ax.set_xlim(xmin-0.02*(xmax-xmin), xmax+0.02*(xmax-xmin))
    ax.set_ylim(ymin-0.02*(ymax-ymin), ymax+0.02*(ymax-ymin))
    ax.set_aspect('equal', adjustable='box'); ax.axis('off')
    fig.tight_layout(); fig.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close(fig)
    return out_png


In [ ]:
data_dir =  r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
tick = 494
fp = os.path.join(data_dir, f"weighted_matrix_with_coords_combined_tick_{tick}.csv")
# Define the output directory for network plots"network_plots2
output_dir = os.path.join(data_dir, "network_plots2")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
out_full = os.path.join(output_dir, f"spatial_network_tick_{tick}.png")
plot_spatial_network(fp, out_full, mode='topk_out', k=3, abs_thresh=0.0,
                     title=f"Spatial network (tick {tick})")

# Zoom region in coordinate units (adjust)
region = (5, 55, 5, 55)
out_zoom = os.path.join(output_dir, f"spatial_network_tick_{tick}_zoom.png")
plot_zoom(fp, out_zoom, region=region)



